In [1]:
from bizlogic.portfolio_helper import create_performance_report_only
from bizlogic.portfolio_performance import get_portfolio_performance_v2
from bizlogic.portfolio_db import prepare_raw_holdings_from_db
from typing import Dict
from sqlalchemy import func, and_, or_
import json
from datetime import date, timedelta
from utils.finalyca_store import get_finalyca_scoped_session
import pandas as pd
import numpy as np
from utils.utils import print_query
from fin_models.masters_models import *
from fin_models.transaction_models import *
# from bizlogic.analytics import get_nav_trend, get_nav
import datetime
import time

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
db_session = get_finalyca_scoped_session(False)

In [4]:
account_ids = [
    43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 76, 77, 78, 79, 80, 81, 82, 83, 
    84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]
portfolio_date = datetime.date(2022, 4, 30)
detailed_analysis = False

# account_ids = [8, 9, 10, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 89, 90, 91, 92]
# portfolio_date = datetime.date(2022, 8, 31)
# detailed_analysis = False

account_ids = [116, 117]
portfolio_date = datetime.date(2022, 11, 30)

holdings = prepare_raw_holdings_from_db(db_session, account_ids, portfolio_date, detailed_analysis)

In [25]:
start_date = datetime.date(2022, 1, 31)
end_date = datetime.date(2023, 1, 31)

start_t = time.time()
results = get_portfolio_performance_v2(db_session, holdings, start_date, end_date)
end_t = time.time()

print(F"Time taken with new method: {round(end_t-start_t, 4)} seconds.")

Time taken with new method: 0.1158 seconds.


In [26]:
results

,isin,name,units,start_date,start_unit_price,end_date,end_unit_price,start_price,end_price,change_pr
0,INF846K01W80,AXIS Gold ETF,3200.000,2022-01-31,4139.4700,2023-01-31,4841.1400,1.324630e+07,1.549165e+07,16.950721
1,INF179KC1981,HDFC Gold ETF,3200.000,2022-01-31,4252.0700,2023-01-31,4984.6700,1.360662e+07,1.595094e+07,17.229255
2,INF174KA1HJ8,Kotak GOLD ETF,4000.000,2022-01-31,4149.3400,2023-01-31,4933.6000,1.659736e+07,1.973440e+07,18.900837
3,INF200KA16D8,SBI Gold ETF,3600.000,2022-01-31,4259.3900,2023-01-31,5063.6800,1.533380e+07,1.822925e+07,18.882751
4,INF789F1AUX7,UTI-Gold ETF,4000.000,2022-01-31,4145.3100,2023-01-31,4923.9300,1.658124e+07,1.969572e+07,18.783155
5,INF754K01BS2,Edelweiss Balanced Advantage Fund - Dir (G),1796.525,2022-01-31,39.0500,2023-01-31,39.9600,7.015430e+04,7.178914e+04,2.330346
6,INF179K01WA6,HDFC Balanced Advantage Fund - Direct (G),218.706,2022-01-31,302.7740,2023-01-31,344.1490,6.621849e+04,7.526745e+04,13.665308
7,INF109K012B0,ICICI Pru Balanced Advantage Fund - Dir (G),1253.219,2022-01-31,53.9700,2023-01-31,57.7100,6.763623e+04,7.232327e+04,6.929776
8,INF204K01G94,Nippon India Balanced Advantage Fund - Direct (G),518.853,2022-01-31,131.3806,2023-01-31,138.5121,6.816722e+04,7.186742e+04,5.428123
9,INF966L01580,Quant Multi Asset Fund - Direct (G),838.440,2022-01-31,81.1380,2023-01-31,90.2330,6.802934e+04,7.565496e+04,11.209298


In [27]:
response = dict()
if results is not None:
    oldvalue = results["start_price"].sum()
    newvalue = results["end_price"].sum()
    gain = ((newvalue-oldvalue)/oldvalue)*100

    # response["performance"] = results.to_dict(orient="records")
    response["equity_old_value"] = oldvalue
    response["equity_new_value"] = newvalue
    response["equity_performance"] = gain

In [28]:
response

{'equity_old_value': 75705537.58429581,
 'equity_new_value': 89468862.23382531,
 'equity_performance': 18.180076502599903}